# Lesson 3: Enhancing Transactions with Unwatch

# Transitioning to `UNWATCH`

Welcome back! Previously, you learned about using the `WATCH` command to implement atomic transactions in Redis. This powerful feature allows you to monitor keys and ensure updates are made safely when specific conditions are met. Now, we'll build upon that knowledge and introduce the `UNWATCH` command. This will give you even more control over your transactions by allowing you to cancel the effects of a `WATCH`.

## What You'll Learn

In this lesson, you'll dive into enhancing transaction control using the `UNWATCH` command. Specifically, you will learn:

- 🔍 **Using `UNWATCH` to Cancel Monitored Keys**: How to stop monitoring keys when certain conditions within your transaction are not met.
- 🔄 **Implementing Conditional Updates with `UNWATCH`**: Writing functions that ensure changes are only made when valid and safe to do so.

Let's walk through a practical example to make this concept clearer:

## Practical Example

```python
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

def update_balance(user_id, increment):
    with client.pipeline() as pipe:
        while True:
            try:
                pipe.watch(f'balance:{user_id}')
                balance = int(pipe.get(f'balance:{user_id}') or 0)
                pipe.multi()
                if balance + increment < 0:  # Prevent negative balances
                    pipe.unwatch()
                    break
                pipe.set(f'balance:{user_id}', balance + increment)
                pipe.execute()
                break
            except redis.WatchError:
                continue

client.set('balance:1', 100)
update_balance(1, 50)
print(f"Updated balance for user 1: {client.get('balance:1')}")
update_balance(1, -200)  # This will not succeed due to the negative balance check
update_balance(1, -50)  # This will succeed
print(f"Final balance for user 1: {client.get('balance:1')}")
```

In this example, you can see `pipe.unwatch()` being used within the `update_balance` function. This ensures that if our condition to prevent a negative balance is not met, the transaction monitoring is canceled.

## Why Use `UNWATCH`?

### 1. 🛡️ **Preventing Negative Balances**
We want to ensure that the user's balance does not go below zero. If the increment would result in a negative balance, we cancel the transaction by calling `unwatch()`. You might ask, "Why not just break out of the loop?" The reason is that breaking out of the loop would not cancel the monitoring of the key, which could lead to unexpected behavior in subsequent transactions, especially in a multi-threaded environment.

### 2. 🧑‍💻 **Code Readability**
Using `unwatch()` makes the code more readable and explicit. It clearly communicates that the transaction is being canceled due to a specific condition not being met. Although, in simple cases like this, breaking out of the loop would work, using `unwatch()` is a good practice to ensure the transaction is canceled correctly and consistently.

## Why It Matters

- ✅ **Enhanced Control**: By using `unwatch()`, you can safely exit transactions without making unwanted changes when certain conditions are not favorable.
- 🚫 **Prevention of Errors**: It helps prevent unintended updates, such as deducting more funds than available in an account, which is critical in financial applications.
- ⚡ **Optimizing Performance**: Utilizing `unwatch()` smartly can help optimize your transaction management, reducing the overhead of unnecessary retries.

Mastery of the `UNWATCH` command will enable you to write more robust applications, ensuring strong control over your data integrity and transaction flow.

## Conclusion

Understanding how and when to use the `UNWATCH` command can significantly enhance your control over Redis transactions. By preventing unwanted changes and ensuring that transactions only succeed when all conditions are met, you can ensure your applications maintain data integrity and perform efficiently.

Are you ready to put this into practice? Let’s move on to the practice section and apply these concepts to become more proficient with Redis transactions!

## Concurrent Transactions with Watch and Unwatch

Great job so far! Now, let's try something different.

This script simulates a scenario where two functions update the same key concurrently: one function using transactions and the watch command, and another function running in a separate thread making simple updates.

By running this code, you will observe how watch helps prevent conflicting updates and ensures data integrity in the presence of concurrent modifications.

Pay careful attention to how the watch and unwatch commands interact, and how they prevent the transaction from executing when the watched key changes.

```py
import threading
import redis
import time

client = redis.Redis(host='localhost', port=6379, db=0)

def update_score(user_id, points):
    with client.pipeline() as pipe:
        while True:
            try:
                pipe.watch(f'score:{user_id}')
                score = int(pipe.get(f'score:{user_id}') or 0)
                if score + points < 0:
                    pipe.unwatch()
                    print(f'Cannot set value {score} + {points} as it is negative')
                    break
                pipe.multi()
                pipe.set(f'score:{user_id}', score + points)
                pipe.execute()
                print(f'Set value in transaction: {score + points}')
                break
            except redis.WatchError as e:
                print(f'Error: {e} Retrying...')
                continue

def update_score_continuously(user_id, points, iterations):
    for i in range(iterations):
        score = int(client.get(f'score:{user_id}') or 0)
        client.set(f'score:{user_id}', score + points)
        print(f'Incremented {score} + {points}')
        time.sleep(0.00001)

client.set('score:1', 150)

# Create and start a thread to update the score continuously
thread = threading.Thread(target=update_score_continuously, args=(1, 50, 50))
thread.start()

update_score(1, -500)

value = client.get('score:1').decode('utf-8')

print(f"Updated score for user 1: {value}")

# Wait for the thread to finish
thread.join()


```

## Adding Transaction Limits

Great work so far! You've learned how to implement watch in Redis for conditional transactions.

Now, let's make things a bit more interesting. Currently, our function update_balance increments the balance without considering if it goes beyond a certain maximum limit.

Change the function so that the balance does not exceed a specified max_balance. If adding the increment would push the balance over this limit, the balance should not be updated.

Follow the TODO comments in the starter code to complete the task.

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

max_balance = 200

def update_balance(user_id, increment):
    with client.pipeline() as pipe:
        while True:
            try:
                pipe.watch(f'balance:{user_id}')
                balance = int(pipe.get(f'balance:{user_id}') or 0)
                pipe.multi()
                # TODO: Add a condition to check if the balance exceeds a maximum limit
                    # If it does, unwatch the key and break the loop

                pipe.set(f'balance:{user_id}', balance + increment)
                pipe.execute()
                break
            except redis.WatchError:
                continue

client.set('balance:1', 180)
update_balance(1, 30)

value = client.get('balance:1').decode('utf-8')
print(f"Updated balance for user 1: {value}")

```

Here’s the modified `update_balance` function, which ensures the balance does not exceed the `max_balance` limit. If adding the increment would push the balance over this limit, the transaction will be canceled.

```py
import redis

# Initialize Redis client
client = redis.Redis(host='localhost', port=6379, db=0)

# Define a maximum balance limit
max_balance = 200

def update_balance(user_id, increment):
    key = f'balance:{user_id}'  # The Redis key for the user's balance
    with client.pipeline() as pipe:
        while True:
            try:
                # Monitor the key for changes
                pipe.watch(key)
                
                # Retrieve the current balance, defaulting to 0 if the key doesn't exist
                balance = int(pipe.get(key) or 0)
                
                # Start the transaction
                pipe.multi()
                
                # TODO: Check if the updated balance exceeds the max_balance
                if balance + increment > max_balance:
                    print(f"Transaction denied: balance would exceed the limit of {max_balance}.")
                    pipe.unwatch()  # Cancel the transaction
                    break
                
                # Update the balance if it's within the limit
                pipe.set(key, balance + increment)
                
                # Execute the transaction
                pipe.execute()
                break
            except redis.WatchError:
                # Retry if another client modified the key
                continue

# Test the function
client.set('balance:1', 180)
update_balance(1, 30)  # This transaction should be denied since it exceeds the max_balance

# Retrieve and print the updated balance
value = client.get('balance:1').decode('utf-8')
print(f"Updated balance for user 1: {value}")
```

### Key Changes:
1. **Condition to check maximum balance**: 
   - `if balance + increment > max_balance`: If this condition is met, `unwatch()` is called to cancel the transaction, and the loop breaks without updating the balance.
   
2. **Output message for exceeding limit**: 
   - A message is printed when the balance exceeds the maximum limit, providing feedback for the user.

### Example Output:
If the initial balance is 180 and you attempt to add 30, the balance update will be canceled because it exceeds the `max_balance` of 200.

```
Transaction denied: balance would exceed the limit of 200.
Updated balance for user 1: 180
```

This ensures the balance never exceeds the defined `max_balance`, maintaining control over transaction limits.

## Enhance Redis Transaction Control

Great job so far!

Now, let's enhance the update_balance function by ensuring that transactions will not proceed if they push the balance above a certain limit or below zero.

Change the function so that the balance does not exceed a specified max_balance and does not go below zero. If adding the increment would push the balance above max_balance or below zero, the function should not update the balance and should return without making any changes

```py
import redis

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

max_balance = 200

# Function using WATCH for conditional updates
def update_balance(user_id, increment):
    with client.pipeline() as pipe:
        while True:
            try:
                pipe.watch(f'balance:{user_id}')
                balance = int(pipe.get(f'balance:{user_id}') or 0)
                pipe.multi()
                # TODO: Add a condition to check if the balance exceeds a maximum limit or goes below 0
                    # If it does, unwatch the key and break the loop without updating the balance
                pipe.set(f'balance:{user_id}', balance + increment)
                pipe.execute()
                break
            except redis.WatchError:
                continue

client.set('balance:1', 100)
update_balance(1, 50)  # This should succeed and update the balance to 150
update_balance(1, -200)  # This should not succeed due to the negative balance check
update_balance(1, 150)  # This should not succeed due to the max balance limit check
update_balance(1, -50)  # This should succeed and update the balance to 100
print(f"Final balance for user 1: {client.get('balance:1')}")
```

## Enhancing Redis Transaction Control

Great work so far! Now, let’s practice using the unwatch command to enhance transaction control in Redis.

Your task is to complete the code that updates the stock of an item and prevents the stock from going negative. When the stock update is invalid (i.e., it would lead to a negative stock), the unwatch command should be used to cancel the transaction.

```py
import redis

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

# Function to update stock levels for an item
def update_stock(item_id, change_in_stock):
    with client.pipeline() as pipe:
        while True:
            try:
                # TODO: Monitor stock levels for the given item with watch command

                stock = int(pipe.get(f'stock:{item_id}') or 0)
                pipe.multi()

                # TODO: If the stock update is invalid (i.e., stock + change_in_stock < 0), cancel the transaction and unwatch the key
                    # Remember to break the loop if the stock update is invalid

                pipe.set(f'stock:{item_id}', stock + change_in_stock)
                pipe.execute()
                break
            except redis.WatchError:
                continue

# Example usage
client.set('stock:123', 20)
update_stock(123, -5)  # This should succeed as (20 - 5) is greater than 0
update_stock(123, -30)  # This should not succeed due to negative stock check (15 - 30)
update_stock(123, 10)  # This should succeed as (15 + 10) is greater than 0
print(f"Final stock for item 123: {client.get('stock:123')}")


```

Here's the updated code with the use of the `unwatch` command to cancel the transaction when the stock update would lead to a negative stock:

```py
import redis

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

# Function to update stock levels for an item
def update_stock(item_id, change_in_stock):
    with client.pipeline() as pipe:
        while True:
            try:
                # Monitor stock levels for the given item
                pipe.watch(f'stock:{item_id}')

                stock = int(pipe.get(f'stock:{item_id}') or 0)
                
                # If the stock update is invalid (stock + change_in_stock < 0), cancel the transaction
                if stock + change_in_stock < 0:
                    print(f"Transaction aborted: Negative stock would result for item {item_id}")
                    pipe.unwatch()  # Cancel the transaction
                    break  # Exit the loop
                
                # Start a transaction
                pipe.multi()

                # Update the stock
                pipe.set(f'stock:{item_id}', stock + change_in_stock)
                
                # Execute the transaction
                pipe.execute()
                break  # Success, break the loop

            except redis.WatchError:
                # WatchError indicates that the watched key has changed, retry the operation
                continue

# Example usage
client.set('stock:123', 20)
update_stock(123, -5)   # This should succeed as (20 - 5) is greater than 0
update_stock(123, -30)  # This should not succeed due to negative stock check (15 - 30)
update_stock(123, 10)   # This should succeed as (15 + 10) is greater than 0
print(f"Final stock for item 123: {client.get('stock:123').decode('utf-8')}")
```

### Key Changes:
1. **`pipe.watch(f'stock:{item_id}')`**: This watches the stock key to detect changes by other clients.
2. **`pipe.unwatch()`**: If the stock update would result in negative stock, this cancels the transaction.
3. **Transaction Handling**: If the stock is valid, the pipeline uses `pipe.multi()` to begin a transaction and then `pipe.set()` to update the stock. `pipe.execute()` commits the transaction.
4. **Retry Logic**: If another process changes the stock during the transaction, a `WatchError` is caught, and the loop retries the transaction.



## Update Player Points Atomically

You're doing great!

Your task is to complete the function to update a player's points in a game, ensuring that the points do not go negative. Use Redis transactions with the watch and unwatch commands to prevent negative points.

```py
import redis

# TODO: Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

def update_points(player_id, points):
    with client.pipeline() as pipe:
        while True:
            try:
                # TODO: Watch the points of the specified player with key 'points:{player_id}'
                
                # TODO: Get the current points of the player or take 0 if it doesn't exist
                
                # TODO: Start a Redis transaction with multi() method
                
                # TODO: Ensure the current points + points do not go negative, if they do, unwatch and break
                
                # TODO: Set the new points (current points + points) for the player using the key 'points:{player_id}' 
                
                # TODO: Execute the transaction
                break
            except redis.WatchError:
                continue

client.set('points:1', 50)
update_points(1, 30)
update_points(1, -100)
update_points(1, -20)
print(f"Final points for player 1: {client.get('points:1')}")

```

Here's the updated code for updating a player's points atomically in Redis while ensuring that the points do not go negative. It uses the `watch` and `unwatch` commands to ensure safe transactions.

```py
import redis

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

def update_points(player_id, points):
    with client.pipeline() as pipe:
        while True:
            try:
                # Watch the points of the specified player
                pipe.watch(f'points:{player_id}')
                
                # Get the current points of the player, or default to 0 if it doesn't exist
                current_points = int(pipe.get(f'points:{player_id}') or 0)

                # If the new points would be negative, cancel the transaction and exit
                if current_points + points < 0:
                    print(f"Transaction aborted: Negative points would result for player {player_id}")
                    pipe.unwatch()  # Cancel the watch
                    break  # Exit the loop

                # Start a Redis transaction
                pipe.multi()
                
                # Set the new points (current points + points)
                pipe.set(f'points:{player_id}', current_points + points)

                # Execute the transaction
                pipe.execute()
                break  # Success, break the loop

            except redis.WatchError:
                # If another process modifies the points during the transaction, retry
                continue

# Example usage
client.set('points:1', 50)  # Setting initial points
update_points(1, 30)        # Should succeed, total will be 80
update_points(1, -100)      # Should fail, as points would go negative
update_points(1, -20)       # Should succeed, total will be 60

# Display final points for player 1
print(f"Final points for player 1: {client.get('points:1').decode('utf-8')}")
```

### Key Features:
1. **`pipe.watch(f'points:{player_id}')`**: Monitors the player's points to ensure no other process changes it during the transaction.
2. **Negative Point Prevention**: If the operation would result in negative points, `pipe.unwatch()` is called to cancel the transaction, and the loop breaks.
3. **Transaction**: The Redis transaction begins with `pipe.multi()` to set the new points and then commits with `pipe.execute()`.
4. **Retry Logic**: If the `WatchError` is raised (indicating that another client modified the points), the transaction is retried.

This setup ensures safe updates to player points, preventing them from going negative.